In [1]:
import numpy as np 
import pandas as pd 
import pydicom
import os
import scipy
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import glob
import pathlib
import imageio
from stl import mesh
import stl
from openmesh import *
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
from skimage import measure, morphology, segmentation
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [3]:
def search_dir_dcm (path):
    '''Recebe arquivo de texto com os diretorios, inverte barras e retorna em forma de lista'''
    files = open(path, 'r')
    files = files.readlines()
    for i in files:
        i.replace(os.sep,"/")
    return files    

In [4]:
# Load the scans in given folder path
def load_scan(path):
    slices = [pydicom.dcmread(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [5]:
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [6]:
def generate_markers(image):
    #Creation of the internal Marker
    marker_internal = image < -400
    marker_internal = segmentation.clear_border(marker_internal)
    marker_internal_labels = measure.label(marker_internal)
    areas = [r.area for r in measure.regionprops(marker_internal_labels)]
    areas.sort()
    if len(areas) > 2:
        for region in measure.regionprops(marker_internal_labels):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       marker_internal_labels[coordinates[0], coordinates[1]] = 0
    marker_internal = marker_internal_labels > 0
    #Creation of the external Marker
    external_a = ndimage.binary_dilation(marker_internal, iterations=10)
    external_b = ndimage.binary_dilation(marker_internal, iterations=55)
    marker_external = external_b ^ external_a
    #Creation of the Watershed Marker matrix
    marker_watershed = np.zeros((512, 512), dtype=np.int)
    marker_watershed += marker_internal * 255
    marker_watershed += marker_external * 128
    
    return marker_internal, marker_external, marker_watershed

In [7]:
def seperate_lungs(image):
    #Creation of the markers as shown above:
    marker_internal, marker_external, marker_watershed = generate_markers(image)
    
    #Creation of the Sobel-Gradient
    sobel_filtered_dx = ndimage.sobel(image, 1)
    sobel_filtered_dy = ndimage.sobel(image, 0)
    sobel_gradient = np.hypot(sobel_filtered_dx, sobel_filtered_dy)
    sobel_gradient *= 255.0 / np.max(sobel_gradient)
    
    #Watershed algorithm
    watershed = morphology.watershed(sobel_gradient, marker_watershed)
    
    #Reducing the image created by the Watershed algorithm to its outline
    outline = ndimage.morphological_gradient(watershed, size=(3,3))
    outline = outline.astype(bool)
    
    #Performing Black-Tophat Morphology for reinclusion
    #Creation of the disk-kernel and increasing its size a bit
    blackhat_struct = [[0, 0, 1, 1, 1, 0, 0],
                       [0, 1, 1, 1, 1, 1, 0],
                       [1, 1, 1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1, 1, 1],
                       [0, 1, 1, 1, 1, 1, 0],
                       [0, 0, 1, 1, 1, 0, 0]]
    blackhat_struct = ndimage.iterate_structure(blackhat_struct, 8)
    #Perform the Black-Hat
    outline += ndimage.black_tophat(outline, structure=blackhat_struct)
    
    #Use the internal marker and the Outline that was just created to generate the lungfilter
    lungfilter = np.bitwise_or(marker_internal, outline)
    #Close holes in the lungfilter
    #fill_holes is not used here, since in some slices the heart would be reincluded by accident
    lungfilter = ndimage.morphology.binary_closing(lungfilter, structure=np.ones((5,5)), iterations=3)
    
    #Apply the lungfilter (note the filtered areas being assigned -2000 HU)
    segmented = np.where(lungfilter == 1, image, -2000*np.ones((512, 512)))
    
    return segmented

_______________________________________________________________________________________________________________________________

In [8]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + list(scan[0].PixelSpacing), dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

In [9]:
def stl_conversor(image, i, threshold, name, patient_name):
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    
    verts, faces = measure.marching_cubes_classic(p, threshold)

    mesh_stl = mesh.Mesh(np.zeros(len(verts[faces]), dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            mesh_stl.vectors[i][j] = verts[f[j],:]
            
    mesh_stl.save('C:/Users/Public/DICOM_Segmented_Watershed/Patients/{}/segmented-{}.stl'.format(patient_name, name))

In [10]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

In [11]:
def absoluteFilePaths(directory):
   for dirpath,_,filenames in os.walk(directory):
       for f in filenames:
           yield os.path.abspath(os.path.join(dirpath, f))

In [12]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [13]:
arquive = search_dir_dcm("C:/Users/Public/DICOM/listaDICOM.txt")

In [14]:
#teste
n=3

#n = len(os.listdir("C:/Users/Public/DICOM_Segmented_Watershed/Patients"))
#if n != 0:
#    n -= 1

In [ ]:
for i in range(n,len(arquive)):
    
    INPUT_FOLDER = arquive[i].replace('\n','')
    patients = os.listdir(INPUT_FOLDER)
    patients.sort()
    
    test_patient_scans = load_scan(INPUT_FOLDER)
    test_patient_images = get_pixels_hu(test_patient_scans)
    
    pix_resampled, spacing = resample(test_patient_images, test_patient_scans, [1,1,1])
    
    patient_name = INPUT_FOLDER.replace('\\','-').replace('C:-Users-Public-DICOM-','')
    
    #Segmenta as imagens e salva .png na pasta DICOM_Segmented_Watershed
    #last_slice = len(os.listdir('C:/Users/Public/DICOM_Segmented_Watershed/Patients/' + patient_name + '/Images'))
    #for slices in range(last_slice, len(test_patient_images)):
    #    test_segmented = seperate_lungs(test_patient_images[slices])
    #    print ("Patient", i, ": Segmented Lung", slices)
    #    plt.imshow(test_segmented, cmap='gray')
    #    mkdir_p('C:/Users/Public/DICOM_Segmented_Watershed/Patients/' + patient_name + '/Images')
    #    plt.savefig('C:/Users/Public/DICOM_Segmented_Watershed/Patients/{}/Images/{}.png'.format(patient_name,  "%03d"%slices))
        
    #Faz gif das imagens segmentadas
    print ("Making gif...", end='')
    images = []
    for filepath in pathlib.Path('C:/Users/Public/DICOM_Segmented_Watershed/Patients/{}/Images'.format(patient_name)).glob('**/*'):
        images.append(imageio.imread(filepath))
    imageio.mimsave('C:/Users/Public/DICOM_Segmented_Watershed/Patients/{}/gif.gif'.format(patient_name), images)
    print(' Done')
    
    #Salva arquivo 3d .stl
    print ("Salving 3D models:")
    print ('Ribs...', end = '')
    stl_conversor(pix_resampled, i, 400, 'ribs', patient_name)
    print (' Done')
    
    segmented_lungs = segment_lung_mask(pix_resampled, False)
    segmented_lungs_fill = segment_lung_mask(pix_resampled, True)
    print ('Full Lung... ', end = '')
    stl_conversor(segmented_lungs,i, 0, 'full_lung', patient_name)
    print ('Done')
    print ('Airways... ',end = '')
    stl_conversor(segmented_lungs_fill - segmented_lungs,i, 0, 'airways', patient_name)
    print('Done')

Making gif... Done
Salving 3D model
Ribs... Done
Full Lung... Done
Airways... Done
Making gif... Done
Salving 3D model
Ribs...